<a href="https://colab.research.google.com/github/lmencisoe/CDA/blob/main/taller%204/Funcion_entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_profiling import ProfileReport
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import joblib
from google.colab import drive
drive.mount('/content/drive')
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score, auc
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import seaborn as sns
import warnings
import re
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [2]:
df_2 = pd.read_json('https://raw.githubusercontent.com/lmencisoe/CDA/main/taller%204/DataSet_Entrenamiento_v2.json')

In [ ]:
json_modelo = pd.read_json()

In [3]:
mejor_modelo = joblib.load('/content/drive/MyDrive/CDA_taller4/pipeline_seleccionado.pkl')

In [48]:
def entrenar(data):
  mejor_modelo = joblib.load('/content/drive/MyDrive/CDA_taller4/pipeline_seleccionado.pkl')
  data['TotalCharges'] = data['TotalCharges'].replace("", 0)
  data['TotalCharges'] = data['TotalCharges'].astype(float)
  X_total = data.drop(['Churn', 'customerID'], axis=1)
  data['target'] = np.where(data['Churn']!= 'Yes', 1, 0)
  X_total = X_total.drop(['target'], axis=1)
  Y_total = data['target'].astype(float)
  X_train, X_test, y_train, y_test = train_test_split(X_total, Y_total, test_size=0.2, random_state=2022)
  cat_features = X_total.select_dtypes(exclude=["float64", "int64"]).columns.to_list()
  numeric_features = X_total.select_dtypes(["float64", "int64"]).columns.to_list()

  categorical_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('encoder', OneHotEncoder())
            ]
        )    

  numeric_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())
            ]
        )

  preprocessor = ColumnTransformer(
            transformers=[
                ('numerical', numeric_transformer, numeric_features),
                ('categorical', categorical_transformer, cat_features)
            ]
        )
  xgb_pipeline = Pipeline(
            [
                ("preprocessor", preprocessor),
                (
                    "classifier", XGBClassifier(),
                ),
            ]
        )
  fit_pipe = xgb_pipeline.fit(X_train, y_train)
  resul1 = " ".join(['AUC anterior modelo train:' , mejor_modelo.score(X_train, y_train).astype(str)])
  resul2 = " ".join(['AUC anterior modelo test:' , mejor_modelo.score(X_test, y_test).astype(str)])
  resul3 = " ".join(['AUC nuevo modelo train:' , fit_pipe.score(X_train, y_train).astype(str)])
  resul4 = " ".join(['AUC nuevo modelo test:' , fit_pipe.score(X_test, y_test).astype(str)])
  return resul1, resul2, resul3, resul4  
  #return X_train


In [49]:
entrenar(df_2)

('AUC anterior modelo train: 0.79625',
 'AUC anterior modelo test: 0.825',
 'AUC nuevo modelo train: 0.8775',
 'AUC nuevo modelo test: 0.825')